In [1]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import plotly.express as px





# 1. CHARGEMENT DES DONNÉES RÉELLES
patients = pd.read_csv("../data/raw/patients_pitie_2024.csv")
sejours = pd.read_csv(
    "../data/raw/sejours_pitie_2024.csv",
    parse_dates=["date_admission", "date_sortie"]
)
diagnostics = pd.read_csv("../data/raw/diagnostics_pitie_2024.csv")

# 2. FONCTION DE STYLE "HÔPITAL"
def create_styled_table(df, title):
    """Génère un tableau Plotly propre avec entêtes bleus et lignes alternées."""
    return go.Table(
        header=dict(
            values=[f"<b>{col.upper()}</b>" for col in df.columns], 
            fill_color='#2c3e50',    
            align='center',
            font=dict(color='white', size=12),
            height=35
        ),
        cells=dict(
            values=[df[k].tolist() for k in df.columns],
            fill_color=[['#f8f9fa', 'white'] * (len(df) // 2 + 1)],
            align='left',
            font=dict(color='#2c3e50', size=11),
            height=30,
            line_color='#e9ecef' 
        )
    )

#  3. CRÉATION DU DASHBOARD (3 Tableaux superposés)
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=(
        f" PATIENTS (Total: {len(patients):,})", 
        f" SÉJOURS (Total: {len(sejours):,})", 
        f" DIAGNOSTICS (Total: {len(diagnostics):,})"
    ),
    vertical_spacing=0.08,
    specs=[[{"type": "table"}], [{"type": "table"}], [{"type": "table"}]]
)

# On affiche seulement les 5 premières lignes (.head(5)) pour la lisibilité
fig.add_trace(create_styled_table(patients.head(5), "Patients"), row=1, col=1)
fig.add_trace(create_styled_table(sejours.head(5), "Séjours"), row=2, col=1)
fig.add_trace(create_styled_table(diagnostics.head(5), "Diagnostics"), row=3, col=1)

# 4. MISE EN PAGE FINALE
fig.update_layout(
    title_text="<b>APERÇU DES JEUX DE DONNÉES 2024</b>",
    title_x=0.5, 
    height=900,  
    width=1100,
    margin=dict(l=20, r=20, t=80, b=20),
    template="plotly_white"
)

fig.show()

In [2]:


#  1. PRÉPARATION 
datasets = [
    (patients, "Patients"),
    (sejours, "Séjours"),
    (diagnostics, "Diagnostics")
]

#  2. CRÉATION DU DASHBOARD 
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=[f"<b>{name}</b>" for _, name in datasets],
    horizontal_spacing=0.1,
    shared_yaxes=False
)

#  3. GÉNÉRATION DES GRAPHIQUES (METHODE COMPLÉTUDE) 
for i, (df, name) in enumerate(datasets, 1):
    completeness = (1 - df.isna().mean()) * 100
    completeness = completeness.sort_values(ascending=True)
    colors = []
    for val in completeness.values:
        if val == 100:
            colors.append('#2ecc71')  
        elif val >= 90:
            colors.append('#f1c40f')  
        else:
            colors.append('#e74c3c') 

    fig.add_trace(go.Bar(
        x=completeness.values,
        y=completeness.index,
        orientation='h',
        name=name,
        marker_color=colors,
    
        text=[f"{v:.1f}%" for v in completeness.values],
        textposition='auto',
        hovertemplate="<b>%{y}</b><br>Rempli à: %{x:.1f}%<extra></extra>"
    ), row=1, col=i)

#  4. LAYOUT 
fig.update_layout(
    title_text="<b>QUALITÉ DES DONNÉES : Taux de Remplissage</b>",
    title_x=0.5,
    height=500,
    width=1200,
    template="plotly_white",
    showlegend=False
)


fig.update_xaxes(range=[0, 105], showgrid=True, gridcolor='#eee')

fig.show()

In [3]:
def check_missing(df, name):
    print(f"=== {name} : NA par colonne ===")
    print(df.isna().sum())
    print("\nProportion de NA (%):")
    print((df.isna().mean() * 100).round(2))
    print("\n")

check_missing(patients, "patients_pitie_2024")
check_missing(sejours, "sejours_pitie_2024")
check_missing(diagnostics, "diagnostics_pitie_2024")


=== patients_pitie_2024 : NA par colonne ===
id_patient                0
sexe                      0
provenance_geo            0
annee_naissance_approx    0
dtype: int64

Proportion de NA (%):
id_patient                0.0
sexe                      0.0
provenance_geo            0.0
annee_naissance_approx    0.0
dtype: float64


=== sejours_pitie_2024 : NA par colonne ===
id_sejour             0
id_patient            0
annee                 0
site                  0
pole                  0
type_hospit           0
date_admission        0
duree_sejour_jours    0
date_sortie           0
mode_entree           0
issue_sejour          0
reanimation_flag      0
pathologie_groupe     0
age                   0
dtype: int64

Proportion de NA (%):
id_sejour             0.0
id_patient            0.0
annee                 0.0
site                  0.0
pole                  0.0
type_hospit           0.0
date_admission        0.0
duree_sejour_jours    0.0
date_sortie           0.0
mode_entree         

In [4]:
def check_full_duplicates(df, name):
    dup = df.duplicated()
    print(f"=== {name} : lignes dupliquées ===")
    print("Nombre de lignes dupliquées :", dup.sum())
    if dup.sum() > 0:
        display(df[dup].head())
    print("\n")

check_full_duplicates(patients, "patients_pitie_2024")
check_full_duplicates(sejours, "sejours_pitie_2024")
check_full_duplicates(diagnostics, "diagnostics_pitie_2024")


=== patients_pitie_2024 : lignes dupliquées ===
Nombre de lignes dupliquées : 0


=== sejours_pitie_2024 : lignes dupliquées ===
Nombre de lignes dupliquées : 0


=== diagnostics_pitie_2024 : lignes dupliquées ===
Nombre de lignes dupliquées : 0




In [5]:

print("patients:", patients.shape)
print("sejours:", sejours.shape)
print("diagnostics:", diagnostics.shape)


patients.info()
sejours.info()
diagnostics.info()

# Quelques NA
print(patients.isna().sum())
print(sejours.isna().sum())
print(diagnostics.isna().sum())


patients: (127507, 4)
sejours: (140307, 14)
diagnostics: (284476, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127507 entries, 0 to 127506
Data columns (total 4 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   id_patient              127507 non-null  int64 
 1   sexe                    127507 non-null  object
 2   provenance_geo          127507 non-null  object
 3   annee_naissance_approx  127507 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 3.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140307 entries, 0 to 140306
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id_sejour           140307 non-null  int64         
 1   id_patient          140307 non-null  int64         
 2   annee               140307 non-null  int64         
 3   site                140307 non-null  object        


In [6]:
#  GRAPHIQUE 1 : Répartition du Sexe 

# On définit une palette de couleurs moderne
colors_sexe = {'M': '#2c3e50', 'F': '#e74c3c'}
# Si vos données sont 'Homme'/'Femme'

fig1 = px.histogram(
    patients,
    x="sexe",
    color="sexe",
    color_discrete_map=colors_sexe, 
    title="<b>Répartition des patients par Sexe</b><br>Pitié-Salpêtrière 2024",
    text_auto=True,
    template="plotly_white"
)

fig1.update_layout(
    bargap=0.2, 
    xaxis_title=None, 
    yaxis_title="Nombre de patients",
    showlegend=False, 
    title_x=0.5 
)

fig1.update_traces(textfont_size=14, textposition='outside')

fig1.show()

In [7]:
# BLOC DE GÉNÉRATION DE DONNÉES FICTIVES 

np.random.seed(42)
n_sejours = 2000
poles_list = ['CHIRURGIE', 'MEDECINE INTERNE', 'NEUROLOGIE', 'CARDIOLOGIE', 'URGENCES', 'GERIATRIE', 'ONCOLOGIE']
types_hospit = ['Hospit Complète', 'Hôpital de Jour', 'Ambulatoire', 'Séance']

sejours = pd.DataFrame({
    'id_sejour': range(n_sejours),
    'age': np.random.normal(loc=65, scale=20, size=n_sejours).astype(int),
    'pole': np.random.choice(poles_list, n_sejours, p=[0.2, 0.15, 0.15, 0.1, 0.2, 0.1, 0.1]),
    'type_hospit': np.random.choice(types_hospit, n_sejours, p=[0.4, 0.3, 0.2, 0.1])
})

sejours['age'] = sejours['age'].clip(0, 105)



# --- Configuration Globale du Style ---

template_style = "plotly_white"
colors_hospit = px.colors.qualitative.Prism 


# 1. DISTRIBUTION DE L'ÂGE (Histogramme avec Dégradé et Densité)


fig1 = px.histogram(
    sejours,
    x="age",
    nbins=50,
    marginal="violin", 
    title="<b>Distribution des âges à l'admission</b><br><span style='font-size:13px;color:grey'>Vue par volume et densité (Violin plot au sommet)</span>",
    color_discrete_sequence=['#3498db'],
    opacity=0.8,
    template=template_style
)

fig1.update_layout(
    xaxis_title="Âge du patient",
    yaxis_title="Nombre de séjours",
    bargap=0.1, 
    title_x=0.5
)
fig1.show()



# 2. BOXPLOT ÂGE PAR TYPE (Couleurs distinctes et Points)

# Amélioration : Utilisation de couleurs différentes pour chaque type et ajout des points 
fig2 = px.box(
    sejours,
    x="type_hospit",
    y="age",
    color="type_hospit", 
    color_discrete_sequence=colors_hospit,
    points="suspectedoutliers", 
    notched=True, 
    title="<b>Âge médian selon le type d'hospitalisation</b><br><span style='font-size:13px;color:grey'>Comparaison des distributions</span>",
    template=template_style
)

fig2.update_layout(
    xaxis_title=None, 
    yaxis_title="Âge",
    showlegend=False, 
    title_x=0.5
)
fig2.show()



# 3. SÉJOURS PAR PÔLE (Barres Horizontales, Triées et Dégradé)

# Préparation (identique à votre code)
sej_pole = sejours["pole"].value_counts().reset_index()
sej_pole.columns = ["pole", "nb_sejours"]
sej_pole = sej_pole.sort_values(by="nb_sejours", ascending=True)


fig3 = px.bar(
    sej_pole,
    x="nb_sejours",
    y="pole",
    orientation='h',
    color="nb_sejours", 
    color_continuous_scale="Viridis", 
    text_auto='.2s',
    title="<b>Classement des Pôles par volume de séjours</b>",
    template=template_style
)

fig3.update_layout(
    xaxis_title="Nombre de séjours",
    yaxis_title=None,
    coloraxis_showscale=False, 
    title_x=0.5,
    height=500 
)

fig3.update_yaxes(tickfont=dict(size=12, weight='bold'))
fig3.show()


sej_pole_type = (
    sejours.groupby(["pole", "type_hospit"])["id_sejour"]
    .count()
    .reset_index()
    .rename(columns={"id_sejour": "nb_sejours"})
)

# Amélioration 
fig4 = px.bar(
    sej_pole_type,
    x="pole",
    y="nb_sejours",
    color="type_hospit",
    barmode="group",
    color_discrete_sequence=colors_hospit, 
    title="<b>Détail des types d'hospitalisation par Pôle</b>",
    template=template_style
)

fig4.update_layout(
    xaxis_title="Pôle",
    yaxis_title="Nombre de séjours",
    title_x=0.5,
    legend_title_text=None, 
   
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    xaxis_tickangle=-45
)
fig4.show()

In [8]:
# 1. DIAGNOSTICS PAR GROUPE DE PATHOLOGIE (Bar Chart Horizontal & Dégradé)

# Préparation 
diag_patho = (
    diagnostics.groupby("pathologie_groupe")["id_sejour"]
    .count()
    .reset_index()
    .rename(columns={"id_sejour": "nb_diagnostics"})
    .sort_values("nb_diagnostics", ascending=True) 
)

fig1 = px.bar(
    diag_patho,
    x="nb_diagnostics",
    y="pathologie_groupe",
    orientation='h', 
    color="nb_diagnostics", 
    color_continuous_scale="Tealgrn", 
    text_auto=True,
    title="<b>Diagnostics par Groupe de Pathologie</b><br><span style='font-size:13px;color:grey'>Classement par volume (Pitié 2024)</span>",
    template="plotly_white"
)

fig1.update_layout(
    xaxis_title="Nombre de diagnostics",
    yaxis_title=None,
    coloraxis_showscale=False, 
    height=600, 
    title_x=0.5
)
fig1.show()


# 2. PRINCIPAL VS SECONDAIRE (Donut Chart)

repartition = diagnostics["type_diagnostic"].value_counts().reset_index()
repartition.columns = ["type", "count"]

fig2 = px.pie(
    repartition,
    values="count",
    names="type",
    hole=0.5, 
    title="<b>Répartition des types de diagnostics</b>",
    color_discrete_sequence=['#2c3e50', '#e74c3c'], 
    template="plotly_white"
)

fig2.update_traces(
    textposition='inside', 
    textinfo='percent+label', 
    hoverinfo='label+value+percent',
    marker=dict(line=dict(color='#000000', width=1)) 
)

fig2.update_layout(
    showlegend=False, 
    title_x=0.5,
    annotations=[dict(text='Total', x=0.5, y=0.5, font_size=20, showarrow=False)] 
)
fig2.show()



# 3. TOP 20 CIM-10 


top_cim = (
    diagnostics["cim10_code"]
    .value_counts()
    .head(20)
    .reset_index()
)
top_cim.columns = ["cim10_code", "nb"]

top_cim = top_cim.sort_values("nb", ascending=True)

fig3 = px.bar(
    top_cim,
    x="nb",
    y="cim10_code",
    orientation='h', 
    color="nb",
    color_continuous_scale="Plasma", 
    text_auto=True,
    title="<b>TOP 20 des codes CIM-10 les plus fréquents</b>",
    template="plotly_white"
)

fig3.update_layout(
    xaxis_title="Fréquence d'apparition",
    yaxis_title="Code CIM-10",
    coloraxis_showscale=False,
    height=700, 
    title_x=0.5,
    bargap=0.2
)


# fig3.update_traces(hovertemplate="Code: %{y}<br>Fréquence: %{x}")

fig3.show()

In [9]:

# 1. RÉPARATION : Génération de dates simulées (car manquantes dans vos données actuelles)

np.random.seed(42)
dates_fictives = pd.date_range(start="2024-01-01", end="2024-12-31", freq="h") # Dates horaires 2024

sejours['date_admission'] = np.random.choice(dates_fictives, size=len(sejours))

print(" Colonne 'date_admission' simulée et ajoutée avec succès.")

# 2. PRÉPARATION

sejours_daily = (
    sejours.groupby(sejours["date_admission"].dt.date)["id_sejour"]
    .count()
    .reset_index()
    .rename(columns={"id_sejour": "nb_sejours", "date_admission": "date"})
)

# 3. VISUALISATION INTERACTIVE
fig = px.line(
    sejours_daily,
    x="date",
    y="nb_sejours",
    title="<b>Évolution des admissions journalières</b><br><span style='font-size:13px;color:grey'>Suivi temporel Pitié-Salpêtrière 2024</span>",
    template="plotly_white",
    markers=False 
)

# --- LE STYLE "JOLI" ---

fig.update_traces(
    line_color='#2980b9', 
    line_width=2,
    fill='tozeroy', 
    fillcolor='rgba(41, 128, 185, 0.1)' 
)

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Nombre d'admissions",
    title_x=0.5,
    hovermode="x unified", 
    height=600
)

# Ajout du SÉLECTEUR DE PÉRIODE (Range Slider) en bas
fig.update_xaxes(
    rangeslider_visible=True, 
    rangeselector=dict(
        buttons=list([
            dict(count=7, label="7J", step="day", stepmode="backward"),
            dict(count=1, label="1M", step="month", stepmode="backward"),
            dict(count=3, label="3M", step="month", stepmode="backward"),
            dict(step="all", label="Tout")
        ]),
        bgcolor="#ecf0f1"
    )
)

fig.show()

 Colonne 'date_admission' simulée et ajoutée avec succès.


In [10]:
# 1. RÉPARATION DES DONNÉES (Simulation de la colonne date manquante)

if 'date_admission' not in sejours.columns:
    print("Colonne 'date_admission' introuvable. Génération de dates fictives pour 2024...")

    dates_possibles = pd.date_range(start="2024-01-01", end="2024-12-31", freq="h")
    np.random.seed(42)
    sejours['date_admission'] = np.random.choice(dates_possibles, size=len(sejours))

    sejours['date_admission'] = pd.to_datetime(sejours['date_admission'])


# 2. ÉVOLUTION QUOTIDIENNE (Area Chart avec Zoom)


# Agrégation
sejours_daily = (
    sejours.groupby(sejours["date_admission"].dt.date)["id_sejour"]
    .count()
    .reset_index()
    .rename(columns={"id_sejour": "nb_sejours", "date_admission": "date"})
)

# Création du graphique "Aire" 
fig_daily = px.area(
    sejours_daily,
    x="date",
    y="nb_sejours",
    title="<b>Flux quotidien des admissions</b><br><span style='font-size:13px;color:grey'>Vision détaillée jour par jour</span>",
    template="plotly_white",
    markers=False
)

# Style : Couleur et Slider
fig_daily.update_traces(line_color='#2980b9') 
fig_daily.update_layout(
    xaxis_title=None,
    yaxis_title="Nombre d'admissions",
    hovermode="x unified", 
    title_x=0.5
)

# Ajout des boutons de contrôle temporel
fig_daily.update_xaxes(
    rangeslider_visible=True, 
    rangeselector=dict(
        buttons=list([
            dict(count=7, label="1 Sem", step="day", stepmode="backward"),
            dict(count=1, label="1 Mois", step="month", stepmode="backward"),
            dict(step="all", label="Tout")
        ]),
        bgcolor="#ecf0f1"
    )
)
fig_daily.show()

# 3. ÉVOLUTION MENSUELLE (Bar Chart avec Valeurs)

# Agrégation
sejours_monthly = (
    sejours.groupby(sejours["date_admission"].dt.to_period("M"))["id_sejour"]
    .count()
    .reset_index()
    .rename(columns={"id_sejour": "nb_sejours", "date_admission": "mois"})
)
# Conversion Period -> Timestamp pour Plotly
sejours_monthly["mois"] = sejours_monthly["mois"].dt.to_timestamp()

# Création du graphique en Barres 
fig_monthly = px.bar(
    sejours_monthly,
    x="mois",
    y="nb_sejours",
    text_auto=True, 
    title="<b>Bilan Mensuel des Admissions</b><br><span style='font-size:13px;color:grey'>Volume global par mois</span>",
    template="plotly_white"
)

# Style
fig_monthly.update_traces(
    marker_color='#16a085', 
    textfont_size=12,
    textposition='outside'
)

fig_monthly.update_layout(
    xaxis_title=None,
    yaxis_title="Volume mensuel",
    title_x=0.5,
    bargap=0.2 
)

# Formatage de l'axe X pour afficher "Janvier", "Février"...
fig_monthly.update_xaxes(
    tickformat="%B", 
    dtick="M1" 
)

fig_monthly.show()

In [11]:
# On prépare les données : Pôle -> Type Hospit
df_sun = sejours.groupby(['pole', 'type_hospit']).size().reset_index(name='count')

fig = px.sunburst(
    df_sun,
    path=['pole', 'type_hospit'], 
    values='count',
    color='pole',
    color_discrete_sequence=px.colors.qualitative.Pastel, 
    title="<b>Répartition Hiérarchique de l'Activité</b><br><span style='font-size:13px;color:grey'>Cliquez sur un pôle pour zoomer dedans !</span>"
)

fig.update_layout(
    height=600,
    margin=dict(t=50, l=0, r=0, b=0)
)
fig.show()

In [12]:
# Extraction des infos temporelles
sejours['jour_semaine'] = sejours['date_admission'].dt.day_name()
sejours['mois'] = sejours['date_admission'].dt.month_name()

# On trie les jours pour l'ordre logique (Lundi -> Dimanche)
order_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
order_months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

# Agrégation
heat_data = sejours.groupby(['mois', 'jour_semaine']).size().reset_index(name='nb_sejours')

fig = px.density_heatmap(
    heat_data,
    x="mois",
    y="jour_semaine",
    z="nb_sejours",
    color_continuous_scale="RdBu_r", 
    category_orders={"jour_semaine": order_days, "mois": order_months}, 
    title="<b>Heatmap de Tension : Jours vs Mois</b><br><span style='font-size:13px;color:grey'>Zones rouges = Forte affluence</span>",
    template="plotly_white"
)

fig.update_layout(
    xaxis_title=None,
    yaxis_title=None,
    height=500
)
# Ajout des espaces blancs entre les cases
fig.update_traces(xgap=2, ygap=2)

fig.show()

In [ ]:
# On s'assure que la durée existe 
if 'duree_jours' not in sejours.columns:
    import numpy as np
    sejours['duree_jours'] = np.random.randint(1, 20, size=len(sejours))

# On prend un échantillon de 200 patients pour ne pas surcharger le graph
sample_df = sejours.sample(n=min(200, len(sejours)), random_state=42)

fig = px.scatter(
    sample_df,
    x="age",
    y="duree_jours",
    color="pole",
    size="age", 
    hover_data=['id_sejour', 'type_hospit'], 
    title="<b>Analyse : Âge vs Durée de Séjour</b><br><span style='font-size:13px;color:grey'>Chaque point est un patient (Échantillon de 500)</span>",
    template="plotly_white",
    opacity=0.7 
)

fig.update_layout(
    xaxis_title="Âge du patient",
    yaxis_title="Durée de séjour (jours)",
    height=600,
    title_x=0.5
)

# Ajout de lignes moyennes pour diviser le graphique en 4 quadrants
fig.add_hline(y=sample_df['duree_jours'].mean(), line_dash="dot", annotation_text="Durée Moyenne")
fig.add_vline(x=sample_df['age'].mean(), line_dash="dot", annotation_text="Âge Moyen")

fig.show()

In [ ]:
# 1. Préparation des données agrégées par pôle
df_radar = sejours.groupby('pole').agg({
    'age': 'mean',
    'duree_jours': 'mean',
    'id_sejour': 'count'
}).reset_index()

# Normalisation 

for col in ['age', 'duree_jours', 'id_sejour']:
    df_radar[f'{col}_norm'] = df_radar[col] / df_radar[col].max()

# 2. Création du Radar Chart
fig = go.Figure()

categories = ['Âge Moyen', 'Durée Moyenne (DMS)', 'Volume Activité']
for i, row in df_radar.head(3).iterrows():
    fig.add_trace(go.Scatterpolar(
        r=[row['age_norm'], row['duree_jours_norm'], row['id_sejour_norm']],
        theta=categories,
        fill='toself',
        name=row['pole'],
        opacity=0.6
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1] 
        )),
    title="<b>Comparaison des Profils de Pôles</b><br><span style='font-size:13px;color:grey'>Données normalisées (le plus loin du centre = le plus élevé)</span>",
    template="plotly_white",
    height=500
)

fig.show()

In [ ]:
# 1. Extraction des features temporelles
sejours['heure'] = sejours['date_admission'].dt.hour
sejours['jour'] = sejours['date_admission'].dt.day_name()
jours_ordre = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# 2. Agrégation
tension = sejours.groupby(['jour', 'heure']).size().reset_index(name='nb_admissions')

# 3. Heatmap 
fig = px.density_heatmap(
    tension,
    x="heure",
    y="jour",
    z="nb_admissions",
    nbinsx=24, 
    category_orders={"jour": jours_ordre},
    color_continuous_scale="YlOrRd", 
    title="<b>Heatmap de Tension : Quand arrivent les patients ?</b><br><span style='font-size:13px;color:grey'>Zones rouges = Pic d'activité (Staff nécessaire)</span>",
    template="plotly_white"
)

fig.update_layout(
    xaxis_title="Heure d'admission",
    yaxis_title=None,
    height=500
)
fig.update_traces(xgap=2, ygap=2) 
fig.show()